In [101]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.linear_model import LogisticRegression
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/brazilian-ecommerce/olist_customers_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_sellers_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_order_reviews_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_order_items_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_products_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_geolocation_dataset.csv
/kaggle/input/brazilian-ecommerce/product_category_name_translation.csv
/kaggle/input/brazilian-ecommerce/olist_orders_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_order_payments_dataset.csv


# Reading in all of the Datasets Given

In [102]:
customers = pd.read_csv('../input/brazilian-ecommerce/olist_customers_dataset.csv')
geolocation = pd.read_csv('../input/brazilian-ecommerce/olist_geolocation_dataset.csv')
orderItem = pd.read_csv('../input/brazilian-ecommerce/olist_order_items_dataset.csv')
orderPayments = pd.read_csv('../input/brazilian-ecommerce/olist_order_payments_dataset.csv')
orderReview = pd.read_csv('../input/brazilian-ecommerce/olist_order_reviews_dataset.csv')
orders = pd.read_csv('../input/brazilian-ecommerce/olist_orders_dataset.csv')
products = pd.read_csv('../input/brazilian-ecommerce/olist_products_dataset.csv')
sellers = pd.read_csv('../input/brazilian-ecommerce/olist_sellers_dataset.csv')
translations = pd.read_csv('../input/brazilian-ecommerce/product_category_name_translation.csv')

In [103]:
allData=[customers,geolocation,orderItem,orderPayments,orderReview,orders,products,sellers,translations]
#for x in allData:
   # print (x.info())

# Data Cleaning

## 1) Merging
Mergin Based on
* order_id
* product_id
* seller_id



In [104]:

dep_on_order_id = orders.merge(orderReview).merge(orderPayments).merge(orderItem)
dep_on_product_id = pd.merge(products,orderItem)
dep_on_seller_id = pd.merge(orderItem, sellers)


# Features to keep [order_id]: 
* Shipping limit date
* Payment type
* Installments
* Review creation date
* Order purchase time stamp
* Order delivery customer date
* Order estimated delivery date
* Review Score
* Price 
* Freight Value
* Order approved at

In [105]:
to_keep = ["order_id","shipping_limit_date","payment_type","payment_installments",
           "review_creation_date","order_purchase_timestamp","order_delivered_customer_date",
          "order_estimated_delivery_date","review_score", "price", "freight_value", "order_approved_at"]
for column in dep_on_order_id.columns: 
    if column not in to_keep:
        dep_on_order_id = dep_on_order_id.drop([column],axis=1)
        

# Fabricated Dataframe

# Adding to Order_ID
* **Adding**:
    * Actual Delivery Time (days) = Purchase Timestamp - Order Delivered Date
    * Estimated Delivery Time (days) = Purchase Timestamp - Order Estimated Delivery Date
    * Delivery Time Difference (days) = Estimated - Actual
    * Review Time Differnece (Days) = Review Creation Date - Order Delivered Date
    * Price to Shipping Ratio (Num) = Price / Freight Value (Shipping Price)

In [106]:
timedifference = dep_on_order_id
timedifference['estimated_delivery_time'] = (pd.to_datetime(timedifference['order_estimated_delivery_date']) - pd.to_datetime(timedifference['order_purchase_timestamp'])).dt.days
timedifference['actual_delivery_time'] = (pd.to_datetime(timedifference['order_delivered_customer_date']) - pd.to_datetime(timedifference['order_purchase_timestamp'])).dt.days
timedifference['delivery_time_difference'] = timedifference['estimated_delivery_time'] - timedifference['actual_delivery_time']
timedifference['review_time_difference'] = (pd.to_datetime(timedifference['review_creation_date']) - pd.to_datetime(timedifference['order_delivered_customer_date'])).dt.days
timedifference['approved_time_difference'] = (pd.to_datetime(timedifference['order_approved_at']) - pd.to_datetime(timedifference['order_purchase_timestamp'])).dt.days
timedifference['price_to_shipping_ratio'] = (timedifference['freight_value'] / timedifference['price'] )
timedifference = timedifference[['order_id','review_score','estimated_delivery_time','actual_delivery_time','delivery_time_difference', 'review_time_difference', 'price_to_shipping_ratio','payment_type','payment_installments']]
true_order_id = timedifference
true_order_id['payment_type'] = true_order_id['payment_type'].replace(['credit_card', 'voucher', 'boleto', 'debit_card'], [0,1,2,3])
true_order_id.isnull().sum()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


order_id                       0
review_score                   0
estimated_delivery_time        0
actual_delivery_time        2471
delivery_time_difference    2471
review_time_difference      2471
price_to_shipping_ratio        0
payment_type                   0
payment_installments           0
dtype: int64

# Features to Keep [product_id]
* product_id
* seller_id
* shipping_limit_date
* product_category_name
* Product_name_length
* product_description_length
* product_photos_qty

In [107]:
to_keep = ["product_id","seller_id","shipping_limit_date","product_category_name",
           "product_name_length","product_description_length","product_photos_qty"]
for column in dep_on_product_id.columns: 
    if column not in to_keep:
        dep_on_product_id = dep_on_product_id.drop([column],axis=1)
        

### Creating a merged DataFrame to Match Products and Reviews

In [108]:
order_2 = pd.merge(orderReview, orderItem)
product_2 = pd.merge(order_2, products)


Product_2 is the main product DataFrame

# Features to Keep [seller_id]
* seller_id
* shipping_limit_date
* seller_state
* seller_city
* price

In [109]:
to_keep = ["seller_id","shipping_limit_date","seller_state",
           "seller_city","price","seller_zip_code_prefix"]
for column in dep_on_seller_id.columns: 
    if column not in to_keep:
        dep_on_seller_id = dep_on_seller_id.drop([column],axis=1)
        

Below is the Created Dataset that contains the location variables

In [110]:
merge1 = orders.merge(orderReview).merge(orderPayments).merge(orderItem) #on order_id
merge2 = pd.merge(merge1, products) #on prod_id
merge3 = pd.merge(merge2, sellers) #on seller_id
merge4 = pd.merge(merge3, customers) #on zip_code
location_df = merge4.merge(geolocation, left_on = 'customer_zip_code_prefix', right_on = 'geolocation_zip_code_prefix')




In [ ]:
dep_on_seller_id["seller_zip_code_prefix"].nunique()
zip_rating = location_df[["seller_zip_code_prefix","review_score"]]
zip_rating = zip_rating.groupby(["seller_zip_code_prefix"]).mean()

### Removing All Null Values

### Order

In [ ]:
dep_on_order_id = dep_on_order_id.dropna(how="any")
dep_on_order_id.isnull().sum()

### Product

In [ ]:
product_2.head()
product_2.rename(columns = {"product_name_lenght":"product_name_len","product_description_lenght":"product_description_len"},inplace = True)

product_2 = product_2.dropna()
product_2.isnull().sum()

Almost 85% of the title and over 50% of the comment message are missing. Since we can't 

### Seller

In [ ]:
dep_on_seller_id.isnull().sum()

# Coorelation Heatmaps

### Order

In [ ]:
review_score_heatmap = true_order_id.corr()['review_score'].to_frame().sort_values('review_score', ascending=False)
review_score_heatmap = review_score_heatmap.drop(['review_score'])
sns.heatmap(review_score_heatmap, annot=True)
plt.title("Order_ID Correaltions to Review Score")
plt.show()


### Product

In [ ]:
review_score_heatmap_pd = product_2.corr()['review_score'].to_frame().sort_values('review_score', ascending=False)
review_score_heatmap_pd = review_score_heatmap_pd.drop(['review_score','product_width_cm','product_length_cm','product_height_cm','product_weight_g','order_item_id'])
sns.heatmap(review_score_heatmap_pd, annot=True)
plt.title("ProductID Correaltions to Reivew Score")
plt.show()


### Location Based Data

In [ ]:
review_sc = location_df.corr()['review_score'].to_frame().sort_values('review_score', ascending=False)
review_sc = review_sc.drop(['review_score','product_width_cm','price','product_name_lenght','product_height_cm','product_length_cm','product_weight_g','payment_value','order_item_id'])
sns.heatmap(review_sc, annot=True)
plt.title("Location Correaltions to Review Score")
plt.show()


# Looking at shipping speed across zipcodes

In [ ]:
reviewAndOrders = pd.merge(orderReview, orders, 'inner').drop_duplicates()
reviewAndOrders['estimated_delivery_time'] = (pd.to_datetime(reviewAndOrders['order_estimated_delivery_date']) - pd.to_datetime(reviewAndOrders['order_purchase_timestamp'])).dt.days
reviewAndOrders['actual_delivery_time'] = (pd.to_datetime(reviewAndOrders['order_delivered_customer_date']) - pd.to_datetime(reviewAndOrders['order_purchase_timestamp'])).dt.days
reviewAndOrders = reviewAndOrders[['review_id','order_id','review_score','order_status','estimated_delivery_time','actual_delivery_time']]

In [ ]:
zipDf = pd.merge(customers, orders, 'left', 'customer_id').drop_duplicates()
for ii in [x for x in zipDf.columns if 'date' in x or 'time' in x]:
    zipDf[ii] = pd.to_datetime(zipDf[ii])
    
zipDf['actual_delivery_time'] = (zipDf.order_delivered_customer_date - zipDf.order_purchase_timestamp).dt.days
zipDf = pd.merge(zipDf, reviewAndOrders[['order_id','review_score']], 'left','order_id').drop_duplicates()
zipDf = zipDf.select_dtypes(exclude='datetime')

zipDf = zipDf.dropna()


In [ ]:
zipGroup = zipDf.groupby('customer_zip_code_prefix')
avgDlvTimeByZip = pd.DataFrame(zipGroup['actual_delivery_time'].mean())
sns.distplot((avgDlvTimeByZip))
# highly skewed. Log transform before labeling speed

In [ ]:
avgDlvTimeByZip['delivery_speed'] = pd.qcut(np.log(avgDlvTimeByZip['actual_delivery_time'] + 1), 4, labels= [0,1,2,3]) # larger the value, slower
avgDlvTimeByZip.delivery_speed.value_counts()

In [ ]:
avgDlvTimeByZip.sample(10)

In [ ]:
geolocation = pd.merge(geolocation, avgDlvTimeByZip, 'left', left_on = 'geolocation_zip_code_prefix', right_on = avgDlvTimeByZip.index.name)
m = {0:'very fast',1:'fast',2:'slow',3:'very slow'}
plt.figure(figsize=(20,20))
for ii in range(4):
    temp = geolocation[geolocation.delivery_speed == ii]
    plt.scatter(temp.geolocation_lat, temp.geolocation_lng, label = m[ii], s = 2)
plt.legend()

# Logistic regression

In [ ]:
true_order_id['review_score'] = [0 if x < 4 else 1 for x in true_order_id['review_score']]
true_order_id['is_late'] = true_order_id.actual_delivery_time - true_order_id.estimated_delivery_time
true_order_id['is_late'] = [0 if x > 0 else 1 for x in true_order_id['is_late']]
true_order_id = true_order_id[['review_score','is_late','price_to_shipping_ratio','review_time_difference']]
true_order_id = true_order_id.dropna()

In [ ]:
X, y = true_order_id.iloc[:, 1:], true_order_id.review_score

In [ ]:
logi = LogisticRegression()
logi.fit(X, y)
logi.score(X, y)

In [ ]:
logi.coef_